In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [3]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))


Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium['EX_h2_e'] = 1000
medium['EX_h2o_e'] = 1000
medium['EX_k_e'] = 0
medium['EX_fru_e'] = 10
medium['EX_mg2_e'] = 0
medium['EX_mn2_e'] = 0
medium['EX_na1_e'] = 0
medium['EX_nh4_e'] = 0
medium['EX_ni2_e'] = 0
medium['EX_pi_e'] = 1000
medium['EX_mobd_e'] = 0
medium['EX_zn2_e'] = 0
medium['EX_ribflv_e'] = 0
medium['EX_btn_e'] = 0
medium['EX_so4_e'] = 0
medium['EX_thm_e'] = 0
medium['EX_ca2_e'] = 0
medium['EX_cl_e'] = 0
medium['EX_co2_e'] = 10
medium['EX_cobalt2_e'] = 0
medium['EX_cu2_e'] = 0
medium['EX_fe2_e'] = 0
medium['EX_fol_e'] = 0
medium['EX_h_e'] = 1000
Cljung_Model.medium = medium
#solution = Cljung_Model.optimize()

In [4]:
#Cljung_Model.reactions.ATPM.bounds = 0, 1000

Cljung_Model.reactions.ACt2r.knock_out()
Cljung_Model.reactions.MTHFD.knock_out()
#Cljung_Model.reactions.PFL.knock_out()
#Cljung_Model.reactions.GLNS.knock_out()
#Cljung_Model.reactions.ALCD2y.knock_out() #keep only with NADH

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})

mthfd2 = Reaction("MTHFD2")
mthfd2.name = "Methylenetetrahydrofolate dehydrogenase (NAD)"

mthfd2.add_metabolites({
    Cljung_Model.metabolites.mlthf_c: -1.0,
    Cljung_Model.metabolites.nad_c: -1.0,
    Cljung_Model.metabolites.methf_c: 1.0,
    Cljung_Model.metabolites.nadh_c: 1.0,
})
print(mthfd2.check_mass_balance())    #ensure correct mass balance
print(mthfd2.bounds)                  #Ensure correct bounds
print(mthfd2.reaction)

Cljung_Model.add_reactions([acT, mthfd2])
Cljung_Model.reactions.MTHFD2.bounds = -1000.0, 1000.0


{}
(0.0, 1000.0)
mlthf_c + nad_c --> methf_c + nadh_c


Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0

In [5]:
Cljung_Model.objective = "BIOMASS_Cl_DSM_WT_46p666M1"

In [6]:
#Cljung_Model.objective = "ATPM"
#Cljung_Model.objective = "EX_ac_e"
#Cljung_Model.objective = "EX_etoh_e"
#Cljung_Model.objective = "EX_but_e"
#Cljung_Model.objective = "EX_btoh_e"
#Cljung_Model.objective = "EX_btd_RR_e"
#print(Cljung_Model.slim_optimize())
#solution = Cljung_Model.optimize()
#Cljung_Model.pfba()
#solution.fluxes[solution.fluxes != 0].index

solution = cobra.flux_analysis.pfba(Cljung_Model)
solution.fluxes[solution.fluxes != 0].index

Index(['EX_h2o_e', 'EX_k_e', 'EX_mg2_e', 'EX_mn2_e', 'EX_nh4_e', 'EX_ni2_e',
       'EX_pi_e', 'EX_mobd_e', 'EX_zn2_e', 'BIOMASS_Cl_DSM_WT_46p666M1',
       ...
       'EAR141x', 'EAR160x', 'MPTGex', 'PRPPS', 'PSCVT', 'TKT2', 'PFL',
       'PGSA181', 'PSSA181', 'MTHFD2'],
      dtype='object', length=384)

In [7]:
Cljung_Model.reactions.PIt2r#.summary()

Reaction identifier,PIt2r
Name,Phosphate reversible transport via symport
Memory address,0x07feb2208e780
Stoichiometry,h_e + pi_e <=> h_c + pi_c H+ + Phosphate <=> H+ + Phosphate
GPR,CLJU_RS06110
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
Cljung_Model.reactions.HYDFDN.bounds = -1000.0, 1000.0

In [9]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_pi_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.PIt2r,
Cljung_Model.reactions.ACt,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,

#Cljung_Model.reactions.ATPS4r,
#Cljung_Model.reactions.RNF,
Cljung_Model.reactions.HYDFDN,

Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD2,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,
    
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
    
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

In [31]:
atp_bal = Reaction("BAL_atp_c")
atp_bal.name = "ATP balancing"
atp_bal.add_metabolites({ 
    WLmodel.metabolites.atp_c: -1.0,
    WLmodel.metabolites.adp_c: 1.0
})
atp_bal

nadh_bal = Reaction("BAL_nadh_c")
nadh_bal.name = "NADH balancing"
nadh_bal.add_metabolites({ 
    WLmodel.metabolites.nadh_c: -1.0,
    WLmodel.metabolites.nad_c: 1.0,
})
nadh_bal

h_bal = Reaction("BAL_h_c")
h_bal.name = "H balancing"
h_bal.add_metabolites({ 
    WLmodel.metabolites.h_e: -1.0,
    WLmodel.metabolites.h_c: 1.0,
})
h_bal

pi_bal = Reaction("BAL_pi_c")
pi_bal.name = "Phosphate balancing"
pi_bal.add_metabolites({ 
    WLmodel.metabolites.pi_c: -1.0,
})
pi_bal

WLmodel.add_reactions([atp_bal, nadh_bal, h_bal, pi_bal])#, fdxo_bal, fdxr_bal])

WLmodel.reactions.BAL_atp_c.bounds = -1000.0, 1000.0
#WLmodel.reactions.BAL_adp_c.bounds = -1000.0, 1000.0
WLmodel.reactions.BAL_nadh_c.bounds = -1000.0, 1000.0
#WLmodel.reactions.BAL_nad_c.bounds = -1000.0, 1000.0
WLmodel.reactions.BAL_h_c.bounds = -1000.0, 1000.0
#WLmodel.reactions.BAL_fdxo_c.bounds = -1000.0, 1000.0
WLmodel.reactions.BAL_pi_c.bounds = -1000.0, 1000.0

Ignoring reaction 'BAL_atp_c' since it already exists.
Ignoring reaction 'BAL_nadh_c' since it already exists.
Ignoring reaction 'BAL_h_c' since it already exists.


In [45]:
medium = WLmodel.medium

medium['EX_co2_e'] = 1
medium['EX_h2_e'] = 1000
medium['EX_co_e'] = 0
medium['EX_h2o_e'] = 1000

WLmodel.medium = medium

In [33]:
WLmodel.medium

{'EX_h_e': 1000.0, 'EX_h2o_e': 1000, 'EX_co_e': 1, 'EX_pi_e': 1000.0}

In [46]:
WLmodel.objective = "EX_ac_e"
WLmodel.objective = "EX_etoh_e"
#WLmodel.objective = "EX_but_e"
#WLmodel.objective = "EX_btoh_e"
#WLmodel.objective = "EX_btd_RR_e"

solution = WLmodel.optimize()

print("Product: ",WLmodel.slim_optimize())
print("ATP: ",WLmodel.reactions.BAL_atp_c.flux)
print("NADH: ",WLmodel.reactions.BAL_nadh_c.flux)
#print("Ferredoxin: ",WLmodel.reactions.BAL_fdxo_c.flux)
print("H+: ",WLmodel.reactions.BAL_h_c.flux)
#print("Phosphate: ",WLmodel.reactions.BAL_pi_c.flux)

WLmodel.optimize()

Product:  0.5
ATP:  0.0
NADH:  -1.0
H+:  1.0


,fluxes,reduced_costs
EX_h2_e,-2.0,-0.0
EX_h_e,-1.0,-0.0
EX_h2o_e,1.5,0.0
EX_co2_e,-1.0,-1.0
EX_co_e,0.0,-1.0
...,...,...
ctfAB_ac,0.0,0.0
ADC,0.0,0.0
ACTt,0.0,0.0
EX_act_e,0.0,0.0


In [16]:
Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WL_core_withBAL.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 1.0, 'EX_h_e': -1.5, 'EX_h2o_e': 0.0, 'EX_co2_e': 0.0, 'EX_co_e': -1.0, 'EX_…

In [17]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [18]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: 1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{'H': 2.0}
(0.0, 1000.0)
aacoa_c + ac_c --> acac_c + accoa_c + h_c


In [19]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{'H': -1.0}
acac_c + h_c --> act_c + co2_c


In [20]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07feb21cb2518
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07feb21cbffd0
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [22]:
WLmodelActn.add_reactions([ctfAB_ac, adc, actT, act_ex])

In [23]:
WLmodelActn.objective = "EX_act_e"

solution = WLmodel.optimize()

print("Product: ",WLmodelActn.slim_optimize())
print("ATP: ",WLmodelActn.reactions.BAL_atp_c.flux)
print("NADH: ",WLmodelActn.reactions.BAL_nadh_c.flux)
print("Ferredoxin: ",WLmodelActn.reactions.BAL_fdxo_c.flux)
print("H+: ",WLmodelActn.reactions.BAL_h_c.flux)
print("Phosphate: ",WLmodelActn.reactions.BAL_pi_c.flux)

WLmodelActn.optimize()

Product:  0.0
ATP:  0.0
NADH:  -2.0


AttributeError: DictList has no attribute or entry BAL_fdxo_c